<a href="https://colab.research.google.com/github/GaganDdhwaj/Divorce-Prediction/blob/main/Divorce_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df=pd.read_csv("divorce_data.csv" , delimiter=";")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["Q1"].value_counts()

In [ ]:
corr = df.corr()['Divorce'].drop('Divorce')
sort_corr=corr.abs().sort_values(ascending=False)[:20]
sort_corr

In [ ]:
sns.barplot(x=sort_corr.index, y=sort_corr)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Absolute Correlation')
plt.show()

In [ ]:
y = df['Divorce']
X = df.drop('Divorce',axis=1)
mutual_info_scores = mutual_info_classif(X, y)

In [ ]:
feature_scores_df = pd.DataFrame({'Feature': X.columns, 'Mutual_Info_Score': mutual_info_scores})

# Sort the DataFrame by scores in descending order
feature_scores_df = feature_scores_df.sort_values(by='Mutual_Info_Score', ascending=False)

# print  sorted feature scores
feature_scores_df.head()

In [ ]:
best =feature_scores_df['Feature'][:15]
df[best]

In [ ]:
mean_0=[]
mean_1=[]

max_0=[]
max_1=[]

med_0=[]
med_1=[]

for i in best.values:
    a=df[i][df['Divorce']==0].mean()
    mean_0.append(np.round(a,4))

    a=df[i][df['Divorce']==1].mean()
    mean_1.append(np.round(a,4))

    a=df[i][df['Divorce']==0].max()
    max_0.append(a)

    a=df[i][df['Divorce']==1].max()
    max_1.append(a)


    a=df[i][df['Divorce']==0].median()
    med_0.append(a)

    a=df[i][df['Divorce']==1].median()
    med_1.append(a)

In [ ]:
com_best=pd.DataFrame()
com_best.index=best.values

In [ ]:
com_best["mean_0"]=mean_0
com_best["mean_1"]=mean_1

com_best["max_0"]=max_0
com_best["max_1"]=max_1

com_best["med_0"]=med_0
com_best["med_1"]=med_1

com_best

In [ ]:
com_best['mean_1'].plot()
com_best['mean_0'].plot()
plt.xlabel("feature")
plt.ylabel("Mean ")
plt.show()

In [ ]:
plt.bar(com_best['med_0']  ,range(len(com_best['med_0']))  )
plt.bar(com_best['med_1']  ,range(len(com_best['med_1']))  )

plt.xlabel("feature")
plt.ylabel("median")
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(np.abs(df[best].corr()), annot = True  )
plt.title ("Correlation data "  )
plt.show()

In [ ]:
X_train ,X_test , y_train , y_test = train_test_split(X[best],y , random_state=33,shuffle=True , test_size=0.5)

In [ ]:
model_Log=LogisticRegression()

In [ ]:
parameters  = {
                 'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                'solver': ['saga','newton-cg', 'sgd','adam'],
                 'max_iter':[50,100,500,1000]

              }
logistic_grid = GridSearchCV(model_Log, parameters, cv=5, scoring='accuracy')

In [ ]:
logistic_grid.fit(X_train,y_train)

In [ ]:
y_pred=logistic_grid.predict(X_test)

In [ ]:
logistic_grid.score(X_test, y_test)

In [ ]:
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('SVM', SVC()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('KNeighborsClassifier', KNeighborsClassifier()))
models.append(('KNeighborsClassifier', RandomForestClassifier()))

In [ ]:
accuracy_=[]

name_col=[]

for i in range(len(models)):
    kfold = KFold(n_splits=15, shuffle=True, random_state=43)
    scor = cross_val_score(models[i][1], X[best], y, cv=kfold,scoring='accuracy')
    accuracy_.append(scor.mean())
    name_col.append(models[i][0])
    print("{} : score {:.4f} ".format(models[i][0]  , np.mean(accuracy_)  ))

In [ ]:
index_acc = accuracy_.index(max(accuracy_))

In [ ]:
print(f"best model : {name_col[index_acc]} is score it {max(accuracy_)}")